In [1]:
import os
import sys
sys.path.append(os.path.abspath('/mnt/e/Projects/Qwen2-VL-Finetune'))

from src.train.data import make_dpo_data_module, DPODataset
from src.train.params import DataArguments, ModelArguments, DPOArguments
from transformers import AutoProcessor, HfArgumentParser
from dataclasses import dataclass, field
from typing import Optional

In [6]:
model_id = "/mnt/e/Projects/Qwen2-VL-Finetune/sepo/models/Qwen2.5-VL-3B-Instruct"
parser = HfArgumentParser(
    (ModelArguments, DataArguments, DPOArguments))

args_list = [
    "--dpo_loss", "sigmoid",
    "--precompute_ref_log_probs", "False",
    "--beta", "0.1",
    "--use_liger", "True",
    "--model_id", "/mnt/e/Projects/Qwen2-VL-Finetune/sepo/models/Qwen2.5-VL-3B-Instruct",  # 替换 $MODEL_NAME 为实际值
    "--data_path", "/mnt/e/Projects/Qwen2-VL-Finetune/sepo/data/RLAIF_Sample/DPO_format_for_Qwen/dpo.json",
    "--image_folder", "/mnt/e/Projects/Qwen2-VL-Finetune/sepo/data/RLAIF_Sample/DPO_format_for_Qwen/images",
    "--remove_unused_columns", "False",
    "--freeze_vision_tower", "False",
    "--freeze_llm", "False",
    "--freeze_merger", "False",
    "--bf16", "True",
    "--fp16", "False",
    "--disable_flash_attn2", "False",
    "--output_dir", "output/test_dpo",
    "--num_train_epochs", "1",
    "--per_device_train_batch_size", "4",  # 替换 $BATCH_PER_DEVICE 为实际值
    "--gradient_accumulation_steps", "4",  # 替换 $GRAD_ACCUM_STEPS 为实际值
    "--image_min_pixels", str(512 * 28 * 28),
    "--image_max_pixels", str(1280 * 28 * 28),
    "--learning_rate", "1e-5",
    "--merger_lr", "1e-5",
    "--vision_lr", "2e-6",
    "--weight_decay", "0.1",
    "--warmup_ratio", "0.03",
    "--lr_scheduler_type", "cosine",
    "--logging_steps", "1",
    "--tf32", "True",
    "--gradient_checkpointing", "True",
    "--report_to", "tensorboard",
    "--lazy_preprocess", "True",
    "--save_strategy", "steps",
    "--save_steps", "200",
    "--save_total_limit", "10",
    "--dataloader_num_workers", "4"
]

model_args, data_args, training_args = parser.parse_args_into_dataclasses(args_list)
model_args, data_args, training_args

(ModelArguments(model_id='/mnt/e/Projects/Qwen2-VL-Finetune/sepo/models/Qwen2.5-VL-3B-Instruct'),
 DataArguments(data_path='/mnt/e/Projects/Qwen2-VL-Finetune/sepo/data/RLAIF_Sample/DPO_format_for_Qwen/dpo.json', lazy_preprocess=True, image_folder='/mnt/e/Projects/Qwen2-VL-Finetune/sepo/data/RLAIF_Sample/DPO_format_for_Qwen/images', image_min_pixels=401408, image_max_pixels=1003520, video_min_pixels=100352, video_max_pixels=602112, image_resized_width=None, image_resized_height=None, video_resized_width=None, video_resized_height=None, fps=1.0),
 DPOArguments(output_dir='output/test_dpo', overwrite_output_dir=False, do_train=False, do_eval=False, do_predict=False, eval_strategy=<IntervalStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=4, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=4, eval_accumulation_steps=None, eval_delay=0, torch_empty_cache_steps=None, learning_rate=1e-05, weight_de

In [7]:
processor = AutoProcessor.from_pretrained(model_id)
dataset_module = make_dpo_data_module(
    model_id=model_id,
    processor=processor,
    data_args=data_args
)

In [9]:
dataset_module['train_dataset']